In [1]:
##################################################################################################
## -- Libs
##################################################################################################

import pandas as pd
import numpy as np
import neat
import gc
import sys
import pickle
import random
from pulp import *
import math

import warnings

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

In [15]:
##################################################################################################
## -- Read in Data
##################################################################################################

# Meta Data
meta = pd.read_csv('../2. Data/meta_data.csv')
feature_types_dict = dict(zip(meta['feature'], meta['feature_type']))
apply_stats_features = meta[meta['apply_stats'] == 1]['feature'].tolist()
modelling_valid_features = meta[meta['modelling_valid'] == 1]['feature'].tolist()
modelling_valid_prev_season_features = meta[meta['modelling_valid_previous_season'] == 1]['feature'].tolist()

# Load Seasons
# years = ['19-20', '20-21', '21-22', '22-23', '23-24']
years = ['21-22', '22-23', '23-24']
data_dict = {year: pd.read_csv(f'../2. Data/{year} FFL.csv') for year in years}
for year in years:
    data_current_season = data_dict[year][[col for col in modelling_valid_features if col in data_dict[year].columns]].copy()
    for col in data_current_season.columns:
        data_current_season[col] = data_current_season[col].astype(feature_types_dict[col])
    data_current_season['name'] = data_current_season['name'].str.replace(' ', '_')
    data_current_season['kickoff_time'] = pd.to_datetime(data_current_season['kickoff_time'], format='%Y-%m-%dT%H:%M:%SZ')
    data_dict[year] = data_current_season
    data_dict[f'ALL_PLAYERS_{year}'] = data_current_season['name'].unique()

col_dict = {year: set(data_dict[year].columns) for year in years}
for year in years:
    print(f"{year} | Missing Columns to 23-24: {col_dict['23-24'] - col_dict[year]} | {data_dict[year].shape}")

21-22 | Missing Columns to 23-24: set() | (25447, 20)
22-23 | Missing Columns to 23-24: set() | (26505, 20)
23-24 | Missing Columns to 23-24: set() | (29725, 20)


In [17]:
##################################################################################################
## -- Apply stats to Current Season
##################################################################################################

def create_player_dataframes(data, apply_stats_features):
    """
    Creates individual player DataFrames with missing rows for earlier game weeks,
    calculates rolling averages, and concatenates them into a single DataFrame.

    Args:
        data (pd.DataFrame): Original dataset containing player data.
        apply_stats_features (list): List of features for which rolling averages are calculated.

    Returns:
        pd.DataFrame: Updated DataFrame with player data.
    """

    # Create a list to store individual player DataFrames
    player_dfs = []

    # Loop through each player in the dataset
    for player in data['name'].unique():
        player_data = data[data['name'] == player].sort_values(by=['GW'])

        # Find the missing game weeks for the player
        gw_values = set(player_data['GW'].unique())
        missing_values = list(set(range(1, 39)) - gw_values)
        missing_values.sort()
        minimum_gw = player_data['GW'].min()
        less_than_min_gw = [gw for gw in missing_values if gw < minimum_gw]
        greater_than_min_gw = [gw for gw in missing_values if gw > minimum_gw]

        # Create missing rows for earlier game weeks
        if missing_values:
            missing_data = pd.DataFrame({
                'name': [player] * len(missing_values),
                'GW': missing_values,
                'player_available': [False] * len(less_than_min_gw) + [True] * len(greater_than_min_gw),
                'team_played': [False] * len(missing_values)
            })
            for column in data.columns:
                if column not in missing_data.columns:
                    missing_data[column] = np.where(missing_data['player_available'], np.nan, 0)
            for feature in apply_stats_features:
                missing_data[feature] = np.nan
                    
            updated_player_data = pd.concat([missing_data, player_data]).reset_index(drop=True)
            updated_player_data['player_available'] = updated_player_data['player_available'].fillna(True)
            updated_player_data['team_played'] = updated_player_data['team_played'].fillna(True)
        else:
            updated_player_data = player_data
            updated_player_data['player_available'] = True
            updated_player_data['team_played'] = True
        
        # Calculate rolling averages for specified features
        for feature in apply_stats_features:
            updated_player_data[f'{feature}_mean_upto_GW'] = updated_player_data[feature].where(updated_player_data['team_played']).expanding().mean().round(2)
            updated_player_data[f'{feature}_rolling_3GW'] = updated_player_data[feature].where(updated_player_data['team_played']).rolling(window=3).mean().round(2)
            updated_player_data[f'{feature}_rolling_5GW'] = updated_player_data[feature].where(updated_player_data['team_played']).rolling(window=5).mean().round(2)
            if feature != 'value':
                updated_player_data[f'{feature}_mean_upto_GW_norm'] = updated_player_data[f'{feature}_mean_upto_GW'] / updated_player_data['value']
                updated_player_data[f'{feature}_rolling_3GW_norm'] = updated_player_data[f'{feature}_rolling_3GW'] / updated_player_data['value']
                updated_player_data[f'{feature}_rolling_5GW_norm'] = updated_player_data[f'{feature}_rolling_5GW']  / updated_player_data['value']

        player_dfs.append(updated_player_data)

    # Convert updated_player_data to int
    updated_player_data['player_available'] = updated_player_data['player_available'].astype(int)
    updated_player_data['team_played'] = updated_player_data['team_played'].astype(int)

    # Concatenate player DataFrames and handle missing values
    data_updated = pd.concat(player_dfs, ignore_index=True).sort_values(['name', 'GW'])
    data_updated = data_updated.fillna(method='ffill')
    data_updated.loc[data_updated['team_played'] == 0, 'total_points'] = 0

    return data_updated

# Apply the function to each year
data_updated_dict = {year: create_player_dataframes(data_dict[year], apply_stats_features) for year in years}
for year in years:
    print(f'{year}: shape = {data_updated_dict[year].shape}')

21-22: shape = (30183, 85)
22-23: shape = (31109, 85)
23-24: shape = (34005, 85)


In [18]:
##################################################################################################
## -- Create Game Week 0 Data
##################################################################################################

def create_player_gw0_summary(data_previous_season, data_current_season, ALL_PLAYERS):
    """
    Creates a summary of player values for the previous season and prepares the dataset for Game Week 0.
    
    Args:
        data_previous_season (pd.DataFrame): DataFrame containing player data for the previous season.
        data_current_season (pd.DataFrame): DataFrame containing player data for the current season.
        ALL_PLAYERS (list): List of player names for the current season.
    
    Returns:
        pd.DataFrame: Updated DataFrame with player data.
    """
    
    # Summary of player values for the previous season
    data_previous_summary = (
        data_previous_season.groupby('name')[modelling_valid_prev_season_features]
        .mean()
        .round(3)
        .rename(columns=lambda col: f'{col}_previous_season_mean')
        .reset_index()
    )
    
    # Dataset for Game Week 0
    gw0_player_data = pd.DataFrame({'name': ALL_PLAYERS, 'GW': 0})
    
    # Join Position, Team, and Player Availability from GW1
    gw0_player_data = gw0_player_data.merge(
        data_current_season[data_current_season['GW'] == 1][['name', 'position', 'team', 'value', 'player_available', 'team_played']],
        how='left', on='name'
    )
    
    # Add remaining columns from data and set them to 0
    for column in data_current_season.columns:
        if column not in gw0_player_data.columns:
            gw0_player_data[column] = 0
    
    # Join GW0 to the main dataset
    data_gw0 = pd.concat([gw0_player_data, data_current_season])
    
    # Join Last Season Value Summary Stats
    data_gw0 = data_gw0.merge(data_previous_summary, how='left', on='name')
    
    # Indicate if the player is new this season
    data_gw0['new_player_this_season'] = data_gw0[f'{modelling_valid_prev_season_features[0]}_previous_season_mean'].isnull().astype(int)
    
    # Fill missing values with -1
    data_gw0.fillna(-1, inplace=True)
    
    # Correct for mixed types in kickoff time
    data_gw0['kickoff_time'] = data_gw0['kickoff_time'].replace(0, pd.NaT)
    
    return data_gw0

# Dictionary mapping each year to the previous year
prev_year = {years[i]: years[i-1] for i in range(1, len(years))}

# Apply the function to each year
data_updated_GW0_dict = {year: create_player_gw0_summary(data_updated_dict[prev_year[year]], data_updated_dict[year], data_dict[f'ALL_PLAYERS_{year}']) for year in years[1:]}
for year in years[1:]:
    print(f'{year}: shape = {data_updated_GW0_dict[year].shape}')

22-23: shape = (31887, 97)
23-24: shape = (34874, 97)


In [20]:
##################################################################################################
## -- One Hot Encode
##################################################################################################

def create_one_hot_encoded_table(data):
    """
    Creates a one-hot encoded DataFrame by converting categorical columns to dummy variables.

    Args:
        data_22_23_updated (pd.DataFrame): DataFrame containing updated player data.
        GW_column_name (str): Name of the game week column (default is 'GW').

    Returns:
        pd.DataFrame: Final DataFrame with one-hot encoded features.
    """

    # Create one-hot encoding for categorical columns
    char_cols = ['position']
    one_hot_encoding_list = [data]

    for col in char_cols:
        one_hot = pd.get_dummies(data[col], prefix=col, prefix_sep='_').astype(int)
        one_hot_encoding_list.append(one_hot)
    
    # Manually create one-hot encoding for game week column
    one_hot_gw = pd.get_dummies(data['GW'], prefix='GW', prefix_sep='_').astype(int)
    one_hot_encoding_list.append(one_hot_gw)
    
    # Concatenate all one-hot encoded features
    data_final = pd.concat(one_hot_encoding_list, axis=1)
    
    # Drop unnecessary position feature
    data_final = data_final.drop('position_0.0', axis=1)
    
    return data_final

# Apply the function to each year 
data_final_dict = {year: create_one_hot_encoded_table(data_updated_GW0_dict[year]) for year in years[1:]}
for year in years[1:]:
    print(f'{year}: shape = {data_final_dict[year].shape}')

22-23: shape = (31887, 140)
23-24: shape = (34874, 140)


In [21]:
##################################################################################################
## -- Player Dict
##################################################################################################

def create_player_dict(data_dict, ALL_PLAYERS_dict):
    """
    Create a dictionary to store player data.

    Args:
        data_dict (dict): Dictionary containing player data for different seasons.
        ALL_PLAYERS_dict (dict): Dictionary containing lists of all players for each season.

    Returns:
        dict: Nested dictionary with player data.
    """
    
    player_dict = {}

    for year, data in data_dict.items():
        print(f'\n{year}')
        player_attributes = data.columns
        player_dict[year] = {}
        ALL_PLAYERS = ALL_PLAYERS_dict[f'ALL_PLAYERS_{year}']
        
        for i, player in enumerate(ALL_PLAYERS):
            sys.stdout.write(f'\r{player}: {(i+1)/len(ALL_PLAYERS)*100:.2f}% - {i+1} of {len(ALL_PLAYERS)}' + ' ' * 100)
            sys.stdout.flush()
            
            player_data = data[data['name'] == player]
            player_data = player_data.sort_values(by=['GW', 'kickoff_time']).groupby('GW').last().reset_index()
            
            # Create a dictionary for each game week
            player_dict[year][player] = {GW: player_data[player_data['GW'] == GW][player_attributes].to_dict(orient='records') for GW in range(39)}
    
    return player_dict

# Create or load player dictionary
condition = True
if condition:
    player_dict = create_player_dict(data_final_dict, data_dict)
    # Save player dictionary
    with open('../2. Data/player_dict.pkl', 'wb') as file:
        pickle.dump(player_dict, file)

# Load player dictionary
with open('../2. Data/player_dict.pkl', 'rb') as file:
    player_dict = pickle.load(file)



22-23
Yago_de_Santiago_Alonso: 100.00% - 777 of 777                                                                                                                  
23-24
Yunus_Konak: 100.00% - 869 of 869                                                                                                                           

In [22]:
##################################################################################################
## -- Create Player and Team class Definitions
##################################################################################################

# Player class definition
class Player:
    # Initialise a player with attributes from a dictionary
    def __init__(self, **kwargs):
        self.random_key = random.randint(1, 10**7)
        for key, value in kwargs.items():
            setattr(self, key, value)

    # Update player attributes for a given game week
    def update_attributes(self, player_dict, GW):
        attributes = player_dict[GW][0]
        for key, value in attributes.items():
            setattr(self, key, value)
    
    # Get a list of player attributes, excluding certain ones
    def get_attributes(self):
        attributes = vars(self)
        exclude_list = ['GW', 'name', 'position', 'team', 'kickoff_time', 'random_key']
        return [value for key, value in attributes.items() if key not in exclude_list]

# Team class definition
class Team:
    # Initialize a team with a budget and empty lists of players and game week players
    def __init__(self):
        # Starting Values
        self.original_budget = 1_000
        self.dynamic_budget = self.original_budget
        self.team_points = 0
        self.bench_boost = False
        self.SQUAD_MAX_POSITIONS = {'GK': 2,'DEF': 5,'MID': 5,'FWD': 3}
        self.PLAYING_FORMATION = {'GK': (1,1),'DEF': (3,5),'MID': (3,5),'FWD': (2,3)}
        # Current Players/Predictions
        self.starting_11 = []
        self.squad = []
        self.captain = None
        self.squad_predictions = {}
        # GW Players
        self.gw_players = {}
        self.gw_squad = {}

    # Add a player to the team if they fit the position and budget constraints
    def add_player(self, player, prediction_dict):
        if (self.can_player_be_added_based_on_position(player) and 
            player.value <= self.dynamic_budget and 
            player not in self.squad):
            self.squad.append(player)
            self.dynamic_budget -= player.value
            self.squad_predictions[player.name] = prediction_dict[player.name]

    # Add Players to Team
    def add_highest_player_to_squad(self, ranked_players, prediction_dict, all_available_players):
        for player_name, _ in ranked_players:
            player = all_available_players[player_name]
            self.add_player(player, prediction_dict)
            if len(self.squad) == 15:
                break

    # Remove the player with the lowest prediction from the team
    def remove_lowest_player_from_squad(self, prediction_dict):
        lowest_player = min(self.squad, key=lambda player: prediction_dict[player.name])
        self.squad.remove(lowest_player)
        self.dynamic_budget += lowest_player.value
        self.squad_predictions.pop(lowest_player.name)
        
    # Update the team's total points
    def update_team_points(self):
        self.team_points += sum(player.total_points for player in self.starting_11)
        self.team_points += self.captain.total_points
        # Use Bench Boost
        if self.select_bench_boost():
            bench_points = self.use_bench_boost()
            self.team_points += bench_points

    # Pick the Top Ranked Player as Team Captain
    def pick_captain(self):
        self.captain = self.starting_11[0]

    # Check if a player can be added based on their position
    def can_player_be_added_based_on_position(self, new_player):
        player_position = new_player.position
        players_of_same_position = [p for p in self.squad if getattr(p, f'position_{player_position}') == 1]
        return len(players_of_same_position) < self.SQUAD_MAX_POSITIONS[player_position]
    
    # Check if we should use Bench Boost
    def select_bench_boost(self):
        # If Bench Boost already been used
        if self.bench_boost:
            return False
        
        # If Bench Boost Prediction for each player in the squad == 1
        total_predictions = sum(prediction[1] for _, prediction in self.squad_predictions.items())
        if total_predictions == len(self.squad_predictions):
            self.bench_boost = True
            return True
        else:
            return False
    
    # Use Bench Boost
    def use_bench_boost(self):
        bench_players = [player for player in self.squad if player not in self.starting_11]
        bench_points = sum(player.total_points for player in bench_players)
        return bench_points

    # Add the current players to the game week players dictionary
    def add_players_to_gw_players_squad(self, GW, prediction_dict):
        self.gw_players[GW] = sorted(
            [(p.name, p.value, prediction_dict[p.name], p.position) for p in self.starting_11],
            key=lambda x: x[2],
            reverse=True
        )
        self.gw_squad[GW] = sorted(
            [(p.name, p.value, prediction_dict[p.name], p.position) for p in self.squad],
            key=lambda x: x[2],
            reverse=True
        )
    # Create Starting Squad using optimization 
    def create_starting_squad(self, ranked_players, prediction_dict, all_available_players):
        # Create the 'prob' variable to contain the problem data
        prob = LpProblem("Fantasy Football Squad Selection", LpMaximize)

        # A dictionary called 'player_vars' is created to contain the referenced Variables
        player_vars = LpVariable.dicts("Player", (i for i in range(len(ranked_players))), cat='Binary')

        # The objective function is added to 'prob' first
        prob += lpSum([ranked_players[i][1][0]*player_vars[i] for i in range(len(ranked_players))]), "Total Rank of the Football Team"

        # Constraints:

        # Each player can be chosen at most once
        for i in range(len(ranked_players)):
            prob += player_vars[i] <= 1, "Only Once Constraint {}".format(i)

        # The total value of the team should be less than or equal to dynamic_budget
        prob += lpSum([all_available_players[ranked_players[i][0]].value*player_vars[i] for i in range(len(ranked_players))]) <= self.dynamic_budget, "Total Value Constraint"

        # Position constraints
        for position in self.SQUAD_MAX_POSITIONS.keys():
            prob += lpSum([player_vars[i] for i in range(len(ranked_players)) if all_available_players[ranked_players[i][0]].position == position]) == self.SQUAD_MAX_POSITIONS[position], "{} Constraint".format(position)

        # The problem is solved using PuLP's choice of Solver
        prob.solve()

        # Add each selected player to the squad
        for v in prob.variables():
            if v.varValue == 1:
                player = all_available_players[ranked_players[int(v.name.split('_')[1])][0]]
                self.squad.append(player)
                self.dynamic_budget -= player.value
                self.squad_predictions[player.name] = prediction_dict[player.name]

    # Optimize the players in the squad
    def select_playing_11(self):
        # Create the 'prob' variable to contain the problem data
        prob = LpProblem("Optimize Football Squad", LpMaximize)

        # A dictionary called 'player_vars' is created to contain the referenced Variables
        player_vars = LpVariable.dicts("Player", (player.name for player in self.squad), cat='Binary')

        # The objective function is added to 'prob' first
        prob += lpSum([self.squad_predictions[player.name][0]*player_vars[player.name] for player in self.squad]), "Total Rank of the Squad"

        # Constraints:

        # Each player can be chosen at most once
        for player in self.squad:
            prob += player_vars[player.name] <= 1, "Only Once Constraint {}".format(player.name)

        # Exactly 11 players must be picked
        prob += lpSum(player_vars.values()) == 11, "Number of Players Constraint"

        # Position constraints
        for position in self.PLAYING_FORMATION.keys():
            min_players, max_players = self.PLAYING_FORMATION[position]
            num_players_in_position = lpSum([player_vars[player.name] for player in self.squad if player.position == position])
            prob += num_players_in_position >= min_players, "{} Min Constraint".format(position)
            prob += num_players_in_position <= max_players, "{} Max Constraint".format(position)

        # The problem is solved using PuLP's choice of Solver
        prob.solve()

        # Each of the variables is printed with its resolved optimum value
        self.starting_11 = []
        for v in prob.variables():
            if v.varValue == 1:
                player_name = v.name.replace('Player_','')
                player = next(player for player in self.squad if player.name.replace('-','_') == player_name)
                self.starting_11.append(player)
        
        # Sort Player: Highest Ranked First
        self.starting_11.sort(key=lambda player: self.squad_predictions[player.name][0], reverse=True)


In [ ]:
##################################################################################################
## -- Setup NEAT structure
##################################################################################################

# -- Setup NEAT structure
def setup_population(config_path, use_saved_genome=False, winner_path=None):
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                            neat.DefaultSpeciesSet, neat.DefaultStagnation,
                            config_path)
    population = neat.Population(config)
    
    if use_saved_genome and winner_path:
        with open(winner_path, 'rb') as f:
            winner = pickle.load(f)
        for genome_id in population.population.keys():
            population.population[genome_id] = winner
    
    population.add_reporter(neat.StdOutReporter(True))
    population.add_reporter(neat.StatisticsReporter())

    return population

# -- Initialise Players
def initialise_players(season, player_dict):
    all_available_players = {}
    for player in player_dict[season]:
        GW=0
        player_to_add = Player(**player_dict[season][player][GW][0])
        all_available_players[player] = player_to_add
    return all_available_players

# -- Rank Players
def rank_players(all_available_players, net):
    prediction_dict = {}
    for name, player in all_available_players.items():
        if player.player_available == 1:
            outputs = net.activate(player.get_attributes())
            prediction_dict[name] = (outputs[0], math.tanh(outputs[1]))
    ranked_players = sorted(prediction_dict.items(), key=lambda item: item[1], reverse=True)
    return ranked_players, prediction_dict,

# -- Update Players
def update_players(all_available_players, player_dict, season, GW):
    for name, player in all_available_players.items():
        player.update_attributes(player_dict[season][name], GW)
    return all_available_players

# -- Main function
def main(use_saved_genome=False, winner_path=None):
    config_path = "../config.txt"
    population = setup_population(config_path, use_saved_genome, winner_path)
    seasons=['22-23', '23-24']

    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            # Set Starting Fitness
            genome.fitness = 0

            # Run Seasons
            teams = {}
            for season in seasons:

                # Setup Team and Players
                teams[season] = Team()
                all_available_players = initialise_players(season, player_dict)

                # Game Week: 0
                ranked_players, prediction_dict, = rank_players(all_available_players, net)
                teams[season].create_starting_squad(ranked_players, prediction_dict, all_available_players)
                teams[season].add_players_to_gw_players_squad(0, prediction_dict)
                teams[season].select_playing_11()
                teams[season].pick_captain()

                # Game Week: 1 - 38
                for GW in range(1, 39):
                    # Play Games in GW
                    all_available_players = update_players(all_available_players, player_dict, season, GW)
                    # Store Points from GW
                    teams[season].update_team_points()
                    # Store Players from GW
                    teams[season].add_players_to_gw_players_squad(GW, prediction_dict)
                    if GW < 38:
                        # Generate Predictions on Players from GW
                        ranked_players, prediction_dict = rank_players(all_available_players, net)
                        # Remove/Add new player
                        teams[season].remove_lowest_player_from_squad(prediction_dict)
                        teams[season].add_highest_player_to_squad(ranked_players, prediction_dict, all_available_players)
                        teams[season].select_playing_11()
                        teams[season].pick_captain()

            # Genome Fitness = Total Team Points after Season  
            genome.fitness = sum([team.team_points for team in teams.values()])

    winner = population.run(eval_genomes, 10)

    with open('../2. Data/eval_genomes_winner.pkl', 'wb') as file:
        pickle.dump(winner, file)

if __name__ == "__main__":
    use_saved_genome = True  
    winner_path = "../2. Data/eval_genomes_winner.pkl"
    main(use_saved_genome, winner_path)


 ****** Running generation 0 ****** 

Population's average fitness: 1739.36000 stdev: 956.18693
Best fitness: 3389.00000 - size: (2, 270) - species 1 - id 43
Average adjusted fitness: 0.506
Mean genetic distance 1.311, standard deviation 0.329
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    93   3389.0    0.506     0
     2    0     7       --       --     0
Total extinctions: 0
Generation time: 1052.496 sec

 ****** Running generation 1 ****** 

Population's average fitness: 2540.43000 stdev: 601.07321
Best fitness: 3412.00000 - size: (2, 269) - species 1 - id 115
Average adjusted fitness: 0.661
Mean genetic distance 1.386, standard deviation 0.261
Population of 100 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    52   3412.0    0.743     0
     2    1    48   2820.0    0.579     0
Total extinctions: 0
Generation time: 678.

In [ ]:
def simulate_season(winner, config, season, player_dict):
    # Create a neural network from the winner genome
    net = neat.nn.FeedForwardNetwork.create(winner, config)

    # Setup Team and Players
    team = Team()
    all_available_players = initialise_players(season, player_dict)

    # Game Week: 0
    ranked_players, prediction_dict = rank_players(all_available_players, net)
    team.create_starting_squad(ranked_players, prediction_dict, all_available_players)
    team.add_players_to_gw_players_squad(0, prediction_dict)
    team.select_playing_11()
    team.pick_captain()


    # Game Week: 1 - 38
    for GW in range(1, 39):
        # Play Games in GW
        all_available_players = update_players(all_available_players, player_dict, season, GW)
        # Store Points from GW
        team.update_team_points()
        # Store Players from GW
        team.add_players_to_gw_players_squad(GW, prediction_dict)
        if GW < 38:
            # Generate Predictions on Players from GW
            ranked_players, prediction_dict = rank_players(all_available_players, net)
            # Remove/Add new player
            team.remove_lowest_player_from_squad(prediction_dict)
            team.add_highest_player_to_squad(ranked_players, prediction_dict, all_available_players)
            team.select_playing_11()
            team.pick_captain()

    # Print the players selected for each game week
    for GW, players in team.gw_players.items():
        print(f"Game Week {GW}:")
        for player in players:
            print(f"  {player}")

    # Print the total points of the team
    print(f"Total Points: {team.team_points}")


In [ ]:
with open('../2. Data/eval_genomes_winner.pkl', 'rb') as file:
    winner = pickle.load(file)

config_path = "../config.txt"
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

# simulate_season(winner, config, '22-23', player_dict)
simulate_season(winner, config, '23-24', player_dict)
